In [4]:
import openai
import pandas as pd
import tqdm
import glob

import warnings
warnings.filterwarnings("ignore")

In [5]:
with open("../apikey/apikey.txt", "r") as f:
    openai.api_key = f.readline().replace("\n", "")

In [6]:
def query_prompt(prompt, max_tokens=4000):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-16k",
        messages=[
            {"role": "system", "content": "You are a professor at a science university and creating a exam for your students."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=0,
    )
    return response["choices"][0]["message"]["content"]

In [7]:
def get_df(fname):
    def f(categories):
        for cat in categories:
            for word in [
                "geology",
                "physics",
                "chemistry",
                "mathematical",
                "biology",
                "astronomy",
                "ecology",
                "genetics",
                "statistics",
                "theoretical"
            ]:
                if word.lower() in cat.lower():
                    return True
            
        return False
    
    def text_preprocess(text):
        return text.replace("===", "\n").replace("==", "\n")

    df = pd.read_parquet(fname)
    df_science = df[df["categories"].apply(f)]
    df_science["text"] = "title: " + df_science["title"] + "\n" + df_science["text"].apply(text_preprocess)
    return df_science.sample(len(df_science)//40)

In [8]:
files = glob.glob("../data/wikipedia/*.parquet")

In [9]:
import time

In [10]:
import pickle

In [11]:
import json

In [12]:
from datetime import datetime as dt
import os

In [13]:
texts = []

In [14]:
import traceback 
batch_size = 1

def make_prompt(series):
    prompt = f"""
You are an expert AI assistant who specializes in answering multiple-choice questions. You may use the context below if it helps you to answer the following multiple-choice question.
The output should be an array of json format, with "prompt" as the question statement, "A," "B," "C," "D," and "E" as choices, "answer" as the answer choice (one of A through E).

Context:
{series['text']}

"""
    return prompt

def f(series):
    if series["A"] != series["A"]:
        if type(series["choices"]) == dict:
            for key in ["A", "B", "C", "D", "E"]:
                series[key] = series["choices"][key]
        elif type(series["choices"] == list):
            for i, key in enumerate(["A", "B", "C", "D", "E"]):
                series[key] = series["choices"][i]
    return series

now_date = dt.now().strftime("%Y%m%d%H%M%S")

first = True
for file in files:
    if os.path.basename(file) in ["all.parquet"]:
        print(f"pass: {file}")
        continue
    df_science = get_df(file)
    
    for i in tqdm.tqdm(range(len(df_science)), desc=file):
        try:
            series = df_science.iloc[i]
            prompt = make_prompt(series)
            text = query_prompt(prompt)
            texts_json = json.loads(text)
            if first:
                print(texts_json)
                first = False
            if type(texts_json) == dict:
                text_json["wiki_id"] = series["id"]
                text_json["original_text"] = series["text"]
                texts.append(text_json)
            else:
                for text_json in texts_json:
                    text_json["wiki_id"] = series["id"]
                    text_json["original_text"] = series["text"]
                    texts.append(text_json)
        except Exception as e:
            print(e)
            traceback.print_exc()
            print(text)
        if i % 20 == 0:
            df_texts = pd.DataFrame(texts)
            df_texts = df_texts.apply(f, axis=1)

            df_texts.to_csv(f"output_gpt3.5_generate/{now_date}.csv")

../data/wikipedia/a.parquet:   2%|███▎                                                                                                                                                                                                              | 1/63 [00:09<09:55,  9.61s/it]

[{'prompt': 'What is an albedo feature in planetary geology?', 'A': 'A large area on the surface of a planet that shows a contrast in brightness or darkness with adjacent areas', 'B': 'A small area on the surface of a planet that shows a contrast in brightness or darkness with adjacent areas', 'C': 'A feature on the surface of a planet that is only visible through space probes', 'D': 'A feature on the surface of a planet that is only visible through optical telescopes', 'E': 'A feature on the surface of a planet that is only visible through ground-based telescopes using adaptive optics', 'answer': 'A'}]


../data/wikipedia/a.parquet:  22%|██████████████████████████████████████████████▍                                                                                                                                                                  | 14/63 [01:32<05:08,  6.30s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_29328/324541624.py", line 39, in <module>
    texts_json = json.loads(text)
  File "/opt/conda/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/conda/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/conda/lib/python3.10/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)
../data/wikipedia/a.parquet:  24%|█████████████████████████████████████████████████▊                                                      

Expecting value: line 1 column 1 (char 0)
Which of the following is NOT a type of assay based on the nature of the assay process?

A) End point assay
B) Kinetic assay
C) High throughput assay
D) Multiplex assay
E) Ligand binding assay

Answer: E) Ligand binding assay


../data/wikipedia/a.parquet:  32%|██████████████████████████████████████████████████████████████████▎                                                                                                                                              | 20/63 [02:13<04:57,  6.93s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_29328/324541624.py", line 39, in <module>
    texts_json = json.loads(text)
  File "/opt/conda/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/conda/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/conda/lib/python3.10/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)
../data/wikipedia/a.parquet:  33%|█████████████████████████████████████████████████████████████████████▋                                  

...

../data/wikipedia/w.parquet:  85%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋                               | 17/20 [01:29<00:16,  5.59s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_29328/324541624.py", line 39, in <module>
    texts_json = json.loads(text)
  File "/opt/conda/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/conda/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/conda/lib/python3.10/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)
../data/wikipedia/w.parquet:  90%|████████████████████████████████████████████████████████████████████████████████████████████████████████

Expecting value: line 1 column 1 (char 0)
Prompt: Who delivered the Witherby Memorial Lecture in 2019?

A) Arthur Landsborough Thomson
B) David Lack
C) H. N. Southern
D) Bob Furness
E) Claire Spottiswoode

Answer: D) Bob Furness


../data/wikipedia/y.parquet:  75%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎                                                    | 3/4 [00:15<00:05,  5.25s/it]Traceback (most recent call last):
  File "/tmp/ipykernel_29328/324541624.py", line 39, in <module>
    texts_json = json.loads(text)
  File "/opt/conda/lib/python3.10/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/opt/conda/lib/python3.10/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/opt/conda/lib/python3.10/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)
../data/wikipedia/y.parquet: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████

Expecting value: line 1 column 1 (char 0)
Which of the following statements is true about the Yukawa Institute for Theoretical Physics?

A) The institute was founded in 1944 by Yoshitaka Mimura.
B) The institute was named after the first Japanese citizen to receive the Nobel Prize in Physics.
C) The institute is located in Hiroshima, Japan.
D) The institute's research areas include non-equilibrium statistical physics and non-linear physics.
E) The institute has had 13 academic positions since its inception in 1952.

Answer: B



../data/wikipedia/z.parquet: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:18<00:00,  6.21s/it]


In [15]:
 df_texts.to_csv(f"output_gpt3.5_generate/{now_date}.csv")

In [16]:

df_texts["answer"].value_counts()

A                                                                                              435
B                                                                                              167
C                                                                                              144
D                                                                                               93
E                                                                                               84
Arturo Zychlinsky                                                                                3
It must have been published within the past 6 calendar years of the year of its nomination.      2
James Ivory                                                                                      2
Discovery of the autoimmune regulator and the AIRE gene (1997)                                   2
A and B                                                                                          1
Theodore C